based on  
https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb

In [1]:
import pandas as pd

In [44]:
data = pd.read_csv('MD_menu.csv')

In [35]:
# 결과 확인
data.head()

,메뉴 이름,카테고리,신메뉴,메뉴 설명,식재료,중량,열량,가격,세트메뉴 가능여부,세트가격
0,맥크리스피 스리라차 마요,버거,True,빠삭한 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함!,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,"좋아하는건 더 많이 즐기시라고, 두 배 이상 커진 1/4 파운드 비프 패티가 두 장...","우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   메뉴 이름      27 non-null     object 
 1   카테고리       27 non-null     object 
 2   신메뉴        27 non-null     bool   
 3   메뉴 설명      27 non-null     object 
 4   식재료        22 non-null     object 
 5   중량         22 non-null     float64
 6   열량         22 non-null     float64
 7   가격         27 non-null     int64  
 8   세트메뉴 가능여부  22 non-null     object 
 9   세트가격       21 non-null     float64
dtypes: bool(1), float64(3), int64(1), object(5)
memory usage: 2.0+ KB


In [46]:
# 데이터프레임의 열 이름 확인
print("데이터프레임의 열 이름:")
print(data.columns)

# 각 열의 데이터 타입 확인
print("\n각 열의 데이터 타입:")
print(data.dtypes)


데이터프레임의 열 이름:
Index(['메뉴 이름 ', '카테고리', '신메뉴', '메뉴 설명 ', '식재료', '중량', '열량', '가격 ',
       '세트메뉴 가능여부', '세트가격 '],
      dtype='object')

각 열의 데이터 타입:
메뉴 이름         object
카테고리          object
신메뉴             bool
메뉴 설명         object
식재료           object
중량           float64
열량           float64
가격             int64
세트메뉴 가능여부     object
세트가격         float64
dtype: object


In [47]:
data.columns = ['menuname', 'category', 'newmenu', 'description', 'ingredients', 'grams', 'kcals', 'burgerprice ','setmenu', 'setprice ']

In [39]:
data.head()

,menuname,category,newmenu,description,ingredients,grams,kcals,burgerprice,setmenu,setprice
0,맥크리스피 스리라차 마요,버거,True,빠삭한 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함!,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,"좋아하는건 더 많이 즐기시라고, 두 배 이상 커진 1/4 파운드 비프 패티가 두 장...","우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [48]:
# 한글과 영어만 남기고 나머지 문자 제거하는 함수 정의
def clean_text(text):
    import re
    return re.sub(r'[^가-힣a-zA-Z\s]', '', str(text))

def clean_text_num(text):
    import re
    return re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))

# 데이터프레임의 특정열에 clean_text 함수 적용
data[['category', 'newmenu', 'description']] = data[['category', 'newmenu', 'description']].map(clean_text)
data['menuname'] = data['menuname'].map(clean_text_num)


In [49]:
# xao remove
def clean_xao(text):
    return text.replace(u'\xa0', u' ')

# 데이터프레임의 특정열에 clean_text 함수 적용
data[['menuname', 'category', 'newmenu', 'description']] = data[['menuname', 'category', 'newmenu', 'description']].map(clean_xao)

In [50]:
data.head()

,menuname,category,newmenu,description,ingredients,grams,kcals,burgerprice,setmenu,setprice
0,맥크리스피 스리라차 마요,버거,True,빠삭한 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛,"난류,우유,대두,밀,토마토,닭고기,쇠고기",289.0,663.0,8600,True,10500.0
1,베토디 스리라차 마요,버거,True,베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함,"난류,우유,대두,밀,돼지고기,토마토,쇠고기",251.0,621.0,7600,True,9500.0
2,맥스파이시 상하이 버거,버거,False,쌀가루가 더해져 더 바삭해진 닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지...,"난류,대두,밀,토마토,닭고기",246.0,501.0,6300,True,8400.0
3,토마토 치즈 비프 버거,버거,True,신선한 토마토와 고소한 치즈버거의 조화,"난류,우유,대두,밀,토마토,쇠고기",200.0,403.0,4600,True,6900.0
4,더블 쿼터파운더 치즈,버거,False,좋아하는건 더 많이 즐기시라고 두 배 이상 커진 파운드 비프 패티가 두 장 그램육...,"우유,대두,밀,토마토,쇠고기",275.0,770.0,8200,True,10400.0


In [51]:
# '버거' 단어가 없는 메뉴 이름에 '버거' 추가 (category가 '버거'인 경우에만)
def add_burger(row):
    if '버거' in row['category'] and '버거' not in row['menuname']:
        return row['menuname'] + ' 버거'
    return row['menuname']

# 'map' 대신 'apply' 사용
data['menuname'] = data.apply(add_burger, axis=1)

In [52]:
data['menuname'].value_counts().index.tolist()

['맥크리스피 스리라차 마요 버거',
 '불고기 버거',
 '후렌치후라이 미디움',
 '후렌치후라이 스몰',
 '탄산 라지',
 '탄산 미디움',
 '햄버거',
 '트리플 치즈버거',
 '더블 치즈버거',
 '치즈버거',
 '베이컨 토마토 디럭스 버거',
 '슈비 버거',
 '슈슈 버거',
 '에그 불고기 버거',
 '베토디 스리라차 마요 버거',
 '더블 불고기 버거',
 '맥치킨 버거',
 '맥치킨 모짜렐라 버거',
 '1955 버거',
 '맥크리스피 클래식 버거',
 '맥크리스피 디럭스 버거',
 '빅맥 버거',
 '쿼터파운더 치즈 버거',
 '더블 쿼터파운더 치즈 버거',
 '토마토 치즈 비프 버거',
 '맥스파이시 상하이 버거',
 '후렌치후라이 라지']

In [53]:
data.to_csv('MD_menu_cleaned.csv', index=False)

In [ ]:
data = pd.read_csv('MD_menu_cleaned.csv')

In [21]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
res = model.predict(
    "Below is a table with information about McDonald's menu. "
    "Return a JSON list with an entry for each column. Each entry should have "
    '{"name": "column name", "description": "column description", "type": "column data type"}'
    f"\n\n{data.head()}\n\nJSON:\n"
)

/tmp/ipykernel_8400/4102784480.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  res = model.predict(


In [22]:
import json

attribute_info = json.loads(res)
attribute_info

[{'name': 'menuname',
  'description': 'Name of the menu item',
  'type': 'string'},
 {'name': 'category',
  'description': 'Category of the menu item',
  'type': 'string'},
 {'name': 'newmenu',
  'description': 'Indicator if the menu item is new',
  'type': 'boolean'},
 {'name': 'description',
  'description': 'Description of the menu item',
  'type': 'string'},
 {'name': 'ingredients',
  'description': 'Ingredients used in the menu item',
  'type': 'string'},
 {'name': 'grams',
  'description': 'Weight of the menu item in grams',
  'type': 'float'},
 {'name': 'kcals',
  'description': 'Calories of the menu item',
  'type': 'float'},
 {'name': 'burgerprice',
  'description': 'Price of the burger item',
  'type': 'integer'},
 {'name': 'setmenu',
  'description': 'Indicator if the menu item is part of a set menu',
  'type': 'boolean'},
 {'name': 'setprice', 'description': 'Price of the set menu', 'type': 'float'}]

In [33]:
data.nunique()[data.nunique() < 40]

TypeError: unhashable type: 'list'

In [24]:
attribute_info[1]["description"] += (
    f". Valid values are {sorted(data['category'].value_counts().index.tolist())}"
)

In [25]:
attribute_info[0]["description"] += (
    f". Valid values are {sorted(data['menuname'].value_counts().index.tolist())}"
)

In [26]:
attribute_info

[{'name': 'menuname',
  'description': "Name of the menu item. Valid values are ['1955 버거', '더블 불고기 버거', '더블 치즈버거', '더블 쿼터파운더 치즈 버거', '맥스파이시 상하이 버거', '맥치킨 모짜렐라 버거', '맥치킨 버거', '맥크리스피 디럭스 버거', '맥크리스피 스리라차 마요 버거', '맥크리스피 클래식 버거', '베이컨 토마토 디럭스 버거', '베토디 스리라차 마요 버거', '불고기 버거', '빅맥 버거', '슈비 버거', '슈슈 버거', '에그 불고기 버거', '치즈버거', '쿼터파운더 치즈 버거', '탄산 라지', '탄산 미디움', '토마토 치즈 비프 버거', '트리플 치즈버거', '햄버거', '후렌치후라이 라지', '후렌치후라이 미디움', '후렌치후라이 스몰']",
  'type': 'string'},
 {'name': 'category',
  'description': "Category of the menu item. Valid values are ['버거 ', '사이드', '음료 ']",
  'type': 'string'},
 {'name': 'newmenu',
  'description': 'Indicator if the menu item is new',
  'type': 'boolean'},
 {'name': 'description',
  'description': 'Description of the menu item',
  'type': 'string'},
 {'name': 'ingredients',
  'description': 'Ingredients used in the menu item',
  'type': 'string'},
 {'name': 'grams',
  'description': 'Weight of the menu item in grams',
  'type': 'float'},
 {'name': 'kcals',
  'description'

In [15]:
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
)

In [16]:
doc_contents = "Detailed description of McDonald's menu"
prompt = get_query_constructor_prompt(doc_contents, attribute_info)
print(prompt.format(query="{query}"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [17]:
chain = load_query_constructor_runnable(
    ChatOpenAI(model="gpt-3.5-turbo", temperature=0), doc_contents, attribute_info
)

In [21]:
chain.invoke({"query": "치킨이 들어간 버거는 뭐가있어?"})

StructuredQuery(query='치킨', filter=None, limit=None)